<a href="https://colab.research.google.com/github/statscol/icesi_faii_final/blob/master/fundamentos_analiticaII_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Taller Fundamentos de Analítica 2

## Jhon Parra - Jhon López

El dataset "TransaccionesBancarias.csv" contiene la historia transaccional de diferentes clientes a los que se les otorgó un crédito bancario. Se trata de las agregaciones de las operaciones diarias de los 3 meses (90 días) previos a la demanda del crédito. Tenemos además la fecha del crédito, el monto prestado, y una indicación de si el cliente pudo cumplir con sus obligaciones de pago o no.


## **Carga de librerías necesarias**





In [30]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import tensorflow as tf



/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## **Lectura de la información**

In [1]:
##READ FROM GITHUB REPO
!git clone https://github.com/statscol/icesi_faii_final


Cloning into 'icesi_faii_final'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 7 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (7/7), done.


In [6]:
datos=pd.read_csv("icesi_faii_final/TransaccionesBancarias.csv")
datos.head()

,ID,dia_1,dia_2,dia_3,dia_4,dia_5,dia_6,dia_7,dia_8,dia_9,dia_10,dia_11,dia_12,dia_13,dia_14,dia_15,dia_16,dia_17,dia_18,dia_19,dia_20,dia_21,dia_22,dia_23,dia_24,dia_25,dia_26,dia_27,dia_28,dia_29,dia_30,dia_31,dia_32,dia_33,dia_34,dia_35,dia_36,dia_37,dia_38,dia_39,...,dia_54,dia_55,dia_56,dia_57,dia_58,dia_59,dia_60,dia_61,dia_62,dia_63,dia_64,dia_65,dia_66,dia_67,dia_68,dia_69,dia_70,dia_71,dia_72,dia_73,dia_74,dia_75,dia_76,dia_77,dia_78,dia_79,dia_80,dia_81,dia_82,dia_83,dia_84,dia_85,dia_86,dia_87,dia_88,dia_89,dia_90,pago,fecha,monto
0,1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-30.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,-3500.00,...,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,-30.0,0.0,0.0,-102.00,0.0,0.0,0.00,0.00,0.0,-488.43,0.0,0.0,0.0,0.0,-20.0,-5743.11,NO,2019-04-16 10:55:50.111,150.0
1,2,20323.83,0.00,0.00,1149.58,-807.37,1240.97,1189.47,-11766.97,0.00,0.00,-277.23,1911.71,-3333.63,1351.12,-994.11,0.00,0.00,-2584.20,1490.1,1497.91,909.02,-2552.57,0.00,0.00,1369.02,-2527.99,1349.96,-2708.84,-948.15,0.00,0.00,1751.82,1523.25,2105.0,1785.60,526.27,0.00,0.00,299.15,...,1490.23,1470.19,1744.23,0.0,0.0,0.0,-270.07,1590.93,-7581.79,-3303.89,1256.7,0.0,0.0,1224.79,-1671.22,1303.49,-3220.97,1509.8,0.0,0.0,-1490.2,1509.8,0.0,0.0,-3034.03,0.0,0.0,1323.34,1323.34,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,NO,2019-04-16 16:15:32.991,100.0
2,3,0.00,0.00,-7.15,-7.15,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-150.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-30.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.0,0.0,0.0,-30.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0,-30.0,0.0,0.00,NO,2019-04-29 17:47:37.833,100.0
3,5,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,-40.34,-138.75,0.00,0.00,0.00,0.00,-7.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.0,0.0,0.0,-7.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0,-7.0,0.0,0.00,NO,2019-04-29 20:28:54.543,100.0
4,6,17.02,188.69,-62.19,288.53,0.00,0.00,-266.58,-6.97,103.49,-41.82,-831.44,0.00,0.00,288.96,87.07,63.04,-630.05,960.54,0.0,0.00,32.29,14.70,-365.65,-261.43,18.39,0.00,0.00,304.50,212.35,27.61,-310.73,524.48,0.00,0.0,-230.03,6.25,84.03,-115.45,395.57,...,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,NO,2019-05-03 14:46:39.028,900.0


## Limpieza de la Información

In [0]:
#cambiar a tipo fecha
datos['fecha']=pd.to_datetime(datos.fecha)


In [29]:
#pago: cambiar a binaria. 1 si no pagó, 0 si pagó
datos['pago_m']=pd.get_dummies(datos.pago)["NO"]
datos.head()

,ID,dia_1,dia_2,dia_3,dia_4,dia_5,dia_6,dia_7,dia_8,dia_9,dia_10,dia_11,dia_12,dia_13,dia_14,dia_15,dia_16,dia_17,dia_18,dia_19,dia_20,dia_21,dia_22,dia_23,dia_24,dia_25,dia_26,dia_27,dia_28,dia_29,dia_30,dia_31,dia_32,dia_33,dia_34,dia_35,dia_36,dia_37,dia_38,dia_39,...,dia_55,dia_56,dia_57,dia_58,dia_59,dia_60,dia_61,dia_62,dia_63,dia_64,dia_65,dia_66,dia_67,dia_68,dia_69,dia_70,dia_71,dia_72,dia_73,dia_74,dia_75,dia_76,dia_77,dia_78,dia_79,dia_80,dia_81,dia_82,dia_83,dia_84,dia_85,dia_86,dia_87,dia_88,dia_89,dia_90,pago,fecha,monto,pago_m
0,1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-30.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,-3500.00,...,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,-30.0,0.0,0.0,-102.00,0.0,0.0,0.00,0.00,0.0,-488.43,0.0,0.0,0.0,0.0,-20.0,-5743.11,NO,2019-04-16 10:55:50.111,150.0,1
1,2,20323.83,0.00,0.00,1149.58,-807.37,1240.97,1189.47,-11766.97,0.00,0.00,-277.23,1911.71,-3333.63,1351.12,-994.11,0.00,0.00,-2584.20,1490.1,1497.91,909.02,-2552.57,0.00,0.00,1369.02,-2527.99,1349.96,-2708.84,-948.15,0.00,0.00,1751.82,1523.25,2105.0,1785.60,526.27,0.00,0.00,299.15,...,1470.19,1744.23,0.0,0.0,0.0,-270.07,1590.93,-7581.79,-3303.89,1256.7,0.0,0.0,1224.79,-1671.22,1303.49,-3220.97,1509.8,0.0,0.0,-1490.2,1509.8,0.0,0.0,-3034.03,0.0,0.0,1323.34,1323.34,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,NO,2019-04-16 16:15:32.991,100.0,1
2,3,0.00,0.00,-7.15,-7.15,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-150.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-30.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.0,0.0,0.0,-30.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0,-30.0,0.0,0.00,NO,2019-04-29 17:47:37.833,100.0,1
3,5,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,-40.34,-138.75,0.00,0.00,0.00,0.00,-7.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.0,0.0,0.0,-7.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0,-7.0,0.0,0.00,NO,2019-04-29 20:28:54.543,100.0,1
4,6,17.02,188.69,-62.19,288.53,0.00,0.00,-266.58,-6.97,103.49,-41.82,-831.44,0.00,0.00,288.96,87.07,63.04,-630.05,960.54,0.0,0.00,32.29,14.70,-365.65,-261.43,18.39,0.00,0.00,304.50,212.35,27.61,-310.73,524.48,0.00,0.0,-230.03,6.25,84.03,-115.45,395.57,...,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,NO,2019-05-03 14:46:39.028,900.0,1
